In [ ]:
@gin.configurable
class FeatureS3PRL:
    def _init_(self, feature, layer=None, device='cuda:0', name=None):
        self.model = getattr(s3prl.hub,feature)()
        self.device = device
        if isinstance(self.device, list):
            self.device = self.device[0]
        if isinstance(self.device, int):
            self.device = 'cuda:{}'.format(self.device)
        self.model = self.model.to(self.device)

        self.layer = layer
        if name is None:
            name = feature
            if layer is not None:
                name += str(layer)
        self.name = name

    def extract(self, x):
        wav, fs = librosa.core.load(x, sr=16000)
        with torch.no_grad():
            wav = torch.from_numpy(wav).unsqueeze(0).to(self.device)
            results = self.model(wav)
            features = results["hidden_states"]
        if self.layer is not None:
            if isinstance(self.layer, int):
                features = features[self.layer][0].detach().cpu().numpy()
            elif isinstance(self.layer, list):
                features = [features[i][0].detach().cpu().numpy() for i in self.layer]
                features = np.stack(features)
                features = np.transpose(features,(1,0,2))
        else:
            features = [li[0].detach().cpu().numpy() for li in features]
            features = np.stack(features)
            features = np.transpose(features,(1,0,2))
        return features